In [ ]:
import pulp
import json
from argparse import ArgumentParser
import numpy as np

In [ ]:
parser = ArgumentParser()
parser.add_argument('--input', type=str, required=True)
parser.add_argument('--output', type=str, required=True)

args = parser.parse_args()


def main

In [43]:
ps = {}
for source, targets in enumerate(scene['relationships']['left']):
    for target, p in targets:
        ps[f'{source},{target}'] = p

In [74]:
left_relations = cal_most_prob_scene(ps, len(scene['objects']))
print(left_relations)
reverse_relation(left_relations)

[[], [0, 2, 3, 4, 5, 6, 7], [0, 3, 4, 6], [0, 4, 6], [0, 6], [0, 2, 3, 4, 6, 7], [0], [0, 2, 3, 4, 6]]


[[1, 2, 3, 4, 5, 6, 7],
 [],
 [1, 5, 7],
 [1, 2, 5, 7],
 [1, 2, 3, 5, 7],
 [1],
 [1, 2, 3, 4, 5, 7],
 [1, 5]]

In [72]:
def cal_most_prob_scene(edges, num_objects, eps = 1e-50):
    indexes = []
    for i in range(num_objects):
        for j in range(num_objects):
            if i != j:
                indexes.append(f'{i},{j}')

    variables = pulp.LpVariable.dict('', indexes, 0, 1, pulp.LpInteger)

    prob = pulp.LpProblem("sceneGraphProblem", pulp.LpMaximize)
    prob += pulp.lpSum([variables[i] * np.log(max(edges[i], eps)) +
                    (1 - variables[i]) * np.log(max(1 - edges[i], eps)) for i in indexes])

    # constriants
    # between two node, there exist one and only one edge
    for i in range(num_objects):
        for j in range(num_objects):
            if i != j:
                prob += variables[f'{i},{j}'] + variables[f'{j},{i}'] == 1

    # The transitivity must be followed
    for i in range(num_objects):
        for j in range(num_objects):
            for k in range(num_objects):
                if i != j and j != k and i != k:
                    prob += variables[f'{i},{j}'] + variables[f'{j},{k}'] - variables[f'{i},{k}'] <= 1
    #%%
    status = prob.solve()

    if status < 0:
        raise ValueError('The problem is unsolvable')

    rels = [[] for _ in range(num_objects)]
    for v in prob.variables():
        if v.varValue:
            edge = list(map(int, v.name.replace('_', '').split(',')))
            rels[edge[0]].append(edge[1])

    return rels

def reverse_relation(rels):
    all_nodes = set(range(len(rels)))

    ops_rels = []
    for source, targets in enumerate(rels):
        ops_rels.append(list(all_nodes.difference(set(targets + [source]))))

    return ops_rels

In [68]:
set([0]) + set([1])

TypeError: unsupported operand type(s) for +: 'set' and 'set'